In [1]:
from fpdf import FPDF
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
import argparse

from synthpop.recipes.starter2 import Starter
from synthpop.census_helpers import Census
import warnings
warnings.filterwarnings('ignore')

sns.set()

# Synthesis flow

DIAGRAM 0 HERE: synthesize all

## 1.1. What we are postprocessing?

DEFINITION HERE: ESTO ES LO QUE VAMOS A ANALIZAR, PERO QUE SIGNIFICA? REPRESENTA ALGO EN PARTICULAR? COMO SE INTERPRETA ESTA TABLA?

In [2]:
def postprocess(state, county, year, path):
    hh = pd.read_csv(path + '/hh_{}_{}_{}.csv'.
                     format(state, county, year), dtype={'serialno': 'str',
                                                         'block_id': 'str',
                                                         'state': 'str',
                                                         'county': 'str',
                                                         'tract': 'str',
                                                         'block_group': 'str'})
    hh.rename(columns={'Unnamed: 0': 'household_id'}, inplace=True)
    hh.set_index('household_id', inplace=True)

    p = pd.read_csv(path + '/p_{}_{}_{}.csv'.
                    format(state, county, year), dtype={'serialno': 'str', })
    p.rename(columns={'Unnamed: 0': 'person_id', 'hh_id': 'household_id'}, inplace=True)
    p.set_index('person_id', inplace=True)

    # Format persons
    p.rename(columns={'AGEP': 'age', 'RAC1P': 'race_id', 'NP': 'persons', 'SPORDER': 'member_id',
                          'RELP': 'relate', 'SEX': 'sex', 'WKHP': 'hours', 'SCHL': 'edu', 'PERNP': 'earning'},
             inplace=True)
    return hh, p

In [6]:
# Here we have a list of synthesized counties for hh and p from the 02 (AK) state
ls 02/

02_290.pdf           hh_02_195_2013.csv*  p_02_110_2013.csv*
hh_02_013_2013.csv*  hh_02_198_2013.csv*  p_02_122_2013.csv*
hh_02_016_2013.csv*  hh_02_220_2013.csv*  p_02_130_2013.csv*
hh_02_020_2013.csv*  hh_02_230_2013.csv*  p_02_150_2013.csv*
hh_02_050_2013.csv*  hh_02_240_2013.csv*  p_02_164_2013.csv*
hh_02_060_2013.csv*  hh_02_261_2013.csv*  p_02_170_2013.csv*
hh_02_068_2013.csv*  hh_02_270_2013.csv*  p_02_180_2013.csv*
hh_02_070_2013.csv*  hh_02_275_2013.csv*  p_02_185_2013.csv*
hh_02_090_2013.csv*  hh_02_282_2013.csv*  p_02_188_2013.csv*
hh_02_100_2013.csv*  hh_02_290_2013.csv*  p_02_195_2013.csv*
hh_02_105_2013.csv*  p_02_013_2013.csv*   p_02_198_2013.csv*
hh_02_110_2013.csv*  p_02_016_2013.csv*   p_02_220_2013.csv*
hh_02_122_2013.csv*  p_02_020_2013.csv*   p_02_230_2013.csv*
hh_02_130_2013.csv*  p_02_050_2013.csv*   p_02_240_2013.csv*
hh_02_150_2013.csv*  p_02_060_2013.csv*   p_02_261_2013.csv*
hh_02_164_2013.csv*  p_02_068_2013.csv*   p_02_270_2013.csv*
hh_02_170_2013.csv*  p_0

In [8]:
# These files looks like...
hh, p = postprocess('02', '013', 2013, '02/')

In [11]:
hh.head(3)

,serialno,RT,puma00,puma10,NP,TYPE,BLD,TEN,VEH,HINCP,...,hh_size,hh_workers,seniors,sf_detached,tenure_mover,cat_id,state,county,tract,block group
household_id,,,,,,,,,,,,,,,,,,,,,
0,2012000184237,H,-9,400,1,1,2.0,4.0,1.0,32500.0,...,one,one,no,yes,rent not recent,19334,02,013,000100,1
1,2012000184237,H,-9,400,1,1,2.0,4.0,1.0,32500.0,...,one,one,no,yes,rent not recent,19334,02,013,000100,1
2,2013000404401,H,-9,400,1,1,2.0,3.0,2.0,33000.0,...,one,one,no,yes,rent not recent,34694,02,013,000100,1


In [12]:
p.head(3)

,serialno,member_id,puma00,puma10,age,JWTR,relate,SCH,edu,sex,...,ESR,HISP,earning,race_id,hispanic,person_age,person_sex,race,cat_id,household_id
person_id,,,,,,,,,,,,,,,,,,,,,
0,2012000184237,1,-9,400,43,12.0,0,1.0,16.0,1,...,1.0,1,31000.0,4,no,35 to 60,male,other,138262,0
1,2012000184237,1,-9,400,43,12.0,0,1.0,16.0,1,...,1.0,1,31000.0,4,no,35 to 60,male,other,138262,1
2,2012000200475,1,-9,400,39,1.0,0,1.0,19.0,1,...,1.0,1,43300.0,9,no,35 to 60,male,other,138262,8


## 1.2. What does these tables represent? 

## Starter 

DIAGRAM HERE: how Starter is instantiated

### Getting `marginals` and `joint distributions` for households and persons 

### 1. The `acs5` subject table 

In [25]:
# census api key
key = os.environ['CENSUS']

# AK state
state = '02'

# 290 county (Yukon-Koyukuk)
county = '290'

In [26]:
# creates starter object
starter = Starter(key, state, county, acsyear=2013)

In [28]:
# what do we get from starter class?
print([m for m in dir(starter) if not m.startswith('__')])

['acsyear', 'c', 'county', 'get_available_geography_ids', 'get_geography_name', 'get_household_joint_dist_for_geography', 'get_household_marginal_for_geography', 'get_num_geographies', 'get_person_joint_dist_for_geography', 'get_person_marginal_for_geography', 'h_acs', 'h_acs_cat', 'h_pums_cols', 'p_acs', 'p_acs_cat', 'p_pums_cols', 'state', 'tract']


This contains a bunch of methods and variables for a synthesized:

In [31]:
# ...state and county pair
starter.state, starter.county

('02', '290')

##### 1. 1. The `Census` object and his methods 

    A. Tract and Block group acs subject tables

       DIAGRAM 1 HERE

As it is shown in the diagram, a first step that occurs when we create the `starter` object is that methods from `Census` constructor are also instantiated. This gives us back some relevant information. First, the `acs` 5 years [estimates tables](https://www.census.gov/data/developers/data-sets/acs-5year.html) by different geographies:

In [34]:
starter.h_acs.head(2)

,NAME,B19001_001E,B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,...,B08201_002E,B08201_003E,B08201_004E,B08201_005E,B08201_006E,B08202_001E,B08202_002E,B08202_003E,B08202_004E,B08202_005E
0,"Block Group 1, Census Tract 1, Yukon-Koyukuk C...",187,52,13,20,11,20,12,1,18,...,119,40,16,3,7,187,89,74,21,2
1,"Block Group 2, Census Tract 1, Yukon-Koyukuk C...",336,68,19,42,20,11,27,8,9,...,214,71,30,5,13,336,160,133,37,3


In [105]:
# when we instantiate the Starter class to create starter object we are also creating a census object
c = Census(key, acsyear=2013)

In [104]:
starter.c 

In [49]:
c

In [47]:
# that has his own methods 
print([ m for m in dir(c) if not m.startswith('__')])

['_get_fips_lookup', '_get_pums_relationship', '_query', '_read_csv', '_scale_and_merge', 'acsyear_files', 'base_url', 'block_group_and_tract_query', 'block_group_query', 'c', 'download_household_pums', 'download_population_pums', 'fips_df', 'fips_url', 'pums00_household_base_url', 'pums00_population_base_url', 'pums10_household_base_url', 'pums10_population_base_url', 'pums_cache', 'pums_household_state_base_url', 'pums_population_state_base_url', 'pums_relationship_df', 'pums_relationship_file_url', 'tract_query', 'tract_to_puma', 'try_fips_lookup']


##### 1. 2. Articulating `Census` and  `Starter` methods

As we said, a first thing that takes place when creating `starter` is the generation of `block groups` and `tracts` tables for a [group of variables](https://github.com/UDST/synthpop/blob/0bc36f8fef036913b416e0d4eb8a3fda79fc70ad/synthpop/recipes/starter2.py#L52-L68)

In [106]:
# we create and merge both tables with:
c.block_group_and_tract_query

<bound method Census.block_group_and_tract_query of <synthpop.census_helpers.Census object at 0x7fc1ae2573c8>>

In the above diagram it is shown that, inside the `Starter` class some methods from census object `c` are called. But something else to be pointed out here is that, this last one `c` is also calling other methods from `census` module. 

We are interested in the `acs5` method - which is inside the core.py module imported from `import census` module -. Let's check it:

In [43]:
# imported methods from census module
print([ m for m in dir(c.c) if not m.startswith('__')])

['ALL', '_acs', 'acs', 'acs1', 'acs1dp', 'acs3', 'acs3dp', 'acs5', 'acs5dp', 'session', 'sf1', 'sf3']


Now, we will define some arbitrary variables at block group and tract levels to consume `acs5` subject tables

In [107]:
# block group
presence_of_children_columns = ['B11005_001E', 'B11005_002E', 'B11005_011E']
block_group_size_attr=['B11005_001E'] # this is the total within the block group

# census tract
vehicle_columns = ['B08201_0%02dE' % i for i in range(1, 7)]
tract_size_attr=['B08201_001E'] # this is the total within the tract

In [99]:
tr = c.c.acs5.get(['NAME'] + presence_of_children_columns,
                 geo={'for': "tract:*",
                      'in': 'state: 02 county: 290'}, year=2013)

In [100]:
pd.DataFrame(tr)

,NAME,B11005_001E,B11005_002E,B11005_011E,state,county,tract
0,"Census Tract 1, Yukon-Koyukuk Census Area, Alaska",523.0,140.0,383.0,02,290,000100
1,"Census Tract 2, Yukon-Koyukuk Census Area, Alaska",549.0,155.0,394.0,02,290,000200
2,"Census Tract 3, Yukon-Koyukuk Census Area, Alaska",612.0,251.0,361.0,02,290,000300
3,"Census Tract 4, Yukon-Koyukuk Census Area, Alaska",372.0,151.0,221.0,02,290,000400


In [76]:
bg = c.c.acs5.get(['NAME'] + presence_of_children_columns,
                  geo={'for': "block group:*",
                       'in': 'state: 02 county: 290'}, year=2013)

In [82]:
pd.DataFrame(bg)

,NAME,B11005_001E,B11005_002E,B11005_011E,state,county,tract,block group
0,"Block Group 1, Census Tract 1, Yukon-Koyukuk C...",187.0,58.0,129.0,02,290,000100,1
1,"Block Group 2, Census Tract 1, Yukon-Koyukuk C...",336.0,82.0,254.0,02,290,000100,2
2,"Block Group 1, Census Tract 2, Yukon-Koyukuk C...",208.0,57.0,151.0,02,290,000200,1
3,"Block Group 2, Census Tract 2, Yukon-Koyukuk C...",341.0,98.0,243.0,02,290,000200,2
4,"Block Group 1, Census Tract 3, Yukon-Koyukuk C...",234.0,108.0,126.0,02,290,000300,1
5,"Block Group 2, Census Tract 3, Yukon-Koyukuk C...",190.0,76.0,114.0,02,290,000300,2
6,"Block Group 3, Census Tract 3, Yukon-Koyukuk C...",188.0,67.0,121.0,02,290,000300,3
7,"Block Group 1, Census Tract 4, Yukon-Koyukuk C...",180.0,82.0,98.0,02,290,000400,1
8,"Block Group 2, Census Tract 4, Yukon-Koyukuk C...",192.0,69.0,123.0,02,290,000400,2


Both tables are merged at the highest disaggregation level (block group). To do it `scale and merge` Census method scales down from tract to block group level as it is shown below:

In [102]:
# tract variables
starter.h_acs[['NAME']+vehicle_columns]

,NAME,B08201_001E,B08201_002E,B08201_003E,B08201_004E,B08201_005E,B08201_006E
0,"Block Group 1, Census Tract 1, Yukon-Koyukuk C...",187,119,40,16,3,7
1,"Block Group 2, Census Tract 1, Yukon-Koyukuk C...",336,214,71,30,5,13
2,"Block Group 1, Census Tract 2, Yukon-Koyukuk C...",208,51,73,50,21,11
3,"Block Group 2, Census Tract 2, Yukon-Koyukuk C...",341,83,120,83,34,18
4,"Block Group 1, Census Tract 3, Yukon-Koyukuk C...",234,127,58,35,8,4
5,"Block Group 2, Census Tract 3, Yukon-Koyukuk C...",190,103,47,28,7,3
6,"Block Group 3, Census Tract 3, Yukon-Koyukuk C...",188,102,46,28,7,3
7,"Block Group 1, Census Tract 4, Yukon-Koyukuk C...",180,91,46,32,5,2
8,"Block Group 2, Census Tract 4, Yukon-Koyukuk C...",192,98,50,35,5,3


In [ ]:
def from_fips_to_name(state, county):
    lookup = pd.read_csv(
            'https://s3-us-west-1.amazonaws.com/synthpop-data2/national_county.txt', dtype='str')
    data = lookup[(lookup['State ANSI'] == state) & (lookup['County ANSI'] == county)]
    state_name = data['State'].unique()[0]
    county_name = data['County Name'].unique()[0]
    return state_name, county_name

In [ ]:
from_fips_to_name('02', '290')

In [ ]:
def key():
    return os.environ['CENSUS']

In [ ]:
starter = Starter(key(), 'AK', 'Yukon-Koyukuk Census Area', acsyear=2013)

In [ ]:
starter

In [ ]:
geog = starter.c.try_fips_lookup('AK', 'Yukon-Koyukuk Census Area')
ind = pd.Series([geog[0]], index=["state"])

In [ ]:
geog

In [ ]:
ind

In [ ]:
hh_target = starter.get_household_marginal_for_geography(ind)
p_target = starter.get_person_marginal_for_geography(ind)

In [ ]:
hh_target.columns

In [ ]:
hh_target.columns.names

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
hh_target.iloc[:,0:5]